# Energy labels of Dutch neighborhoods

The [Voorbeeldwoningen 2022](https://www.rvo.nl/onderwerpen/wetten-en-regels-gebouwen/voorbeeldwoningen-bestaande-bouw) study describes the distribution of energy labels per vormfactor range for each dwelling type of the [WoON 2018](https://www.woononderzoek.nl/) study.
The latest release of the [3DBAG](https://3dbag.nl) data set (`2023.10.08`) provides the surface areas for calculating the vormfactor for each Pand.
This work explores the possibility of calculating the vormfactor for each dwelling within a Pand and applying the energy label distributions of the Voorbeeldwoningen 2022 study to estimate the energy label distribution of each neighborhood of the Netherlands.

This repository contains the code for estimating the energy labels and report on the results.

The report can be viewed at: https://3dgi.github.io/wijklabels/report_nl.html

The estimated labels can be downloaded from: https://data.3dgi.xyz/wijklabels. The files with `.fgb` are in the [FlatGeobuf](https://flatgeobuf.org/) format, which can be viewed without download in several GIS applications (e.g. QGIS). For example, to view the neighborhood labels in QGIS, open the file URL (`https://data.3dgi.xyz/wijklabels/labels_neighborhood_geom.fgb`) as a vector data source.

**License** of report and data: https://creativecommons.org/licenses/by/4.0/deed.nl. You are free to share and modify the report and the data, as long as you give appropriate credit to 3DGI, provide a link to the license, and indicate if changes were made.

This project was made by [3DGI](https://3dgi.nl) and commissioned by the [Rijksdienst van Ondernemend Nederland](https://rvo.nl).

In [2]:
from dash import Dash, html, dcc, callback, Output, Input
import geopandas
import plotly.graph_objects as go

COLORS = {"#1a9641": "a++++",
          "#52b151": "a+++",
          "#8acc62": "a++",
          "#b8e17b": "a+",
          "#dcf09e": "a",
          "#ffffc0": "b",
          "#ffdf9a": "c",
          "#febe74": "d",
          "#f69053": "e",
          "#e75437": "f",
          "#d7191c": "g"
          }

neighborhoods = geopandas.read_file("labels_neighborhood_geom.fgb").set_index(["gemeentenaam", "buurtnaam"])
nbhd_long = neighborhoods.drop(columns=["geometry"]).melt(var_name="Energielabel", value_name="Percentage", ignore_index=False).sort_index()
nbhd_long["Percentage"] = nbhd_long["Percentage"] * 100 

app = Dash(__name__)

def plot_buurt(selected_municipality, selected_neighborhood):
    fig = go.Figure(data=[go.Bar(
        x=nbhd_long.loc[(selected_municipality, selected_neighborhood)].Energielabel,
        y=nbhd_long.loc[(selected_municipality, selected_neighborhood)].Percentage,
        marker_color=list(COLORS.keys()),
        hovertemplate = "%{x}: %{y:.1f}%"
    )])
    fig.update_yaxes(range=[0, 60])
    fig.update_layout(title_text=f"{selected_municipality}, {selected_neighborhood}", xaxis_title="Energielabel", yaxis_title="Percentage (%)")
    return fig

@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    [Input('municipalities-dropdown', 'value'), Input('neighborhoods-dropdown', 'value')]
)
def update_graph(selected_municipality, selected_neighborhood):
    fig = plot_buurt(selected_municipality, selected_neighborhood)
    return fig

@app.callback(
    Output('neighborhoods-dropdown', 'options'),
    [Input('municipalities-dropdown', 'value')])
def set_neighborhoods_options(selected_municipality):
    return neighborhoods.index.get_loc_level(selected_municipality, level=0)[1]

@app.callback(
    Output('neighborhoods-dropdown', 'value'),
    [Input('neighborhoods-dropdown', 'options')])
def set_neighborhoods_value(available_options):
    return available_options[0]

app.layout = html.Div([
    html.Div([
        html.Div(
            children=[
                html.H5(children="Municipality:"),
                dcc.Dropdown(
                    id="municipalities-dropdown",
                    options=neighborhoods.index.get_level_values(0).unique(),
                    value="'s-Gravenhage",
                    clearable=False
                ),
            ], style={'padding': 10, 'flex': 1}
        ),
        html.Div(
            children=[
                html.H5(children="Neighborhood:"),
                dcc.Dropdown(
                    id="neighborhoods-dropdown",
                    value="Bezuidenhout-Midden",
                    clearable=False
                ),
            ], style={'padding': 10, 'flex': 1}
        ),
    ], style={'display': 'flex', 'flexDirection': 'row'}),
    dcc.Graph(figure={}, id="controls-and-graph"),
])

if __name__ == '__main__':
    app.run(debug=True)